# Instalando bibliotecas

In [33]:
!pip install -r requirements.txt

In [34]:
!python -m spacy info
!python -m spacy download pt_core_news_lg


============================== Info about spaCy ==============================

spaCy version    3.7.6                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        pt_core_news_lg (3.7.0), en_core_web_sm (3.7.1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [35]:
import spacy
from spacy import displacy
from pymongo import MongoClient
import pandas as pd
import numpy as np
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy.matcher import Matcher
import os
from dotenv import load_dotenv, dotenv_values

In [36]:
load_dotenv()
# print(os.getenv("conn_str"))

True

# Conectando a base do MongoDB

In [37]:
conn_str = os.getenv("conn_str")

try:
    client = MongoClient(conn_str)
    print("Connected to MongoDB successfully!")
except Exception as e:
    print(f"Error: {e}")

Connected to MongoDB successfully!


# Acessando a base e coleção

In [38]:
db = client["Cluster"]
collection = db["Audios_gold"]

db.list_collection_names()

['Audios_gold', 'Audios_embeddings']

# Identificando pelo nome do arquivo

In [39]:
query = {"file_name": '2967602.wav'}
projection = {"transcription": 1, "_id": 0}  # 1 inclui, 0 exclui

for transcricao in collection.find(query, projection):
  print(transcricao)

{'transcription': 'Alô, boa tarde. Boa tarde. Fala com o senhor Clóvis Gentili. Sim, Clóvis, me chamo Beatriz. Fala pra todos. Tudo bem? Tudo bem? Que bom! Você gostaria de falar com o senhor? O senhor seria o responsável da CGC atacadista, né? É bem rápido meu contato, senhor Clóvis, eu preciso só fazer um acompanhamento pra saber como tá a experiência e a satisfação de vocês. Esse acompanhamento dura dois minutinhos no máximo. É feito pelo método de NPS. Tudo bem, tudo bem. Tá. O senhor vai avaliar somente de 0A10A marca Totem. Alguns pontos que eu vou perguntar. Tá. Vou confirmar o CNPJ. CNPJ 19 705 oito três zero 1000 contra quatro dois Ok. Ok. Obrigada pela confirmação, senhor Clóvis. Numa escala de 0 a 10, qual a probabilidade do senhor recomendar a TOTVS a um amigo ou colega hoje? Tem algum comentário que o senhor gostaria de deixar sobre a nota? Não. Então, para a gente aprofundar um pouco mais na melhoria, agora eu vou falar os pontos específicos e peço que o senhor avalie tam

In [40]:
transcricao

{'transcription': 'Alô, boa tarde. Boa tarde. Fala com o senhor Clóvis Gentili. Sim, Clóvis, me chamo Beatriz. Fala pra todos. Tudo bem? Tudo bem? Que bom! Você gostaria de falar com o senhor? O senhor seria o responsável da CGC atacadista, né? É bem rápido meu contato, senhor Clóvis, eu preciso só fazer um acompanhamento pra saber como tá a experiência e a satisfação de vocês. Esse acompanhamento dura dois minutinhos no máximo. É feito pelo método de NPS. Tudo bem, tudo bem. Tá. O senhor vai avaliar somente de 0A10A marca Totem. Alguns pontos que eu vou perguntar. Tá. Vou confirmar o CNPJ. CNPJ 19 705 oito três zero 1000 contra quatro dois Ok. Ok. Obrigada pela confirmação, senhor Clóvis. Numa escala de 0 a 10, qual a probabilidade do senhor recomendar a TOTVS a um amigo ou colega hoje? Tem algum comentário que o senhor gostaria de deixar sobre a nota? Não. Então, para a gente aprofundar um pouco mais na melhoria, agora eu vou falar os pontos específicos e peço que o senhor avalie tam

# NLP Spacy

In [41]:
nlp = spacy.load("pt_core_news_lg")

doc = nlp(transcricao['transcription'])

* Identificando entidades

In [42]:
spacy.displacy.render(doc, style="ent", jupyter=True)

* Identificando possíveis notas

In [43]:
nlp = spacy.load("pt_core_news_lg")
entity_ruler = nlp.add_pipe("entity_ruler", before="ner")

patterns = [
    {
        "label": "NOTA",
        "pattern": [
            {
                'LOWER': {
                    'IN': ['zero', 'um', 'dois', 'três', 'quatro', 'cinco', 'seis', 'sete', 'oito', 'nove', 'dez',
                           'total', 'parcial', 'parcialmente',
                           '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
                }
            }
        ]
    }
]

entity_ruler.add_patterns(patterns)


doc = nlp(transcricao['transcription'])



colors = {"NOTA": "linear-gradient(315deg, #f5d020, #f53803)"}
options = {"ents": ["NOTA"], "colors": colors}


spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

In [44]:
doc = nlp(transcricao['transcription'])
type(doc)

spacy.tokens.doc.Doc

* Separar token/palavra

In [45]:
[token.text for token in doc]

['Alô',
 ',',
 'boa',
 'tarde',
 '.',
 'Boa',
 'tarde',
 '.',
 'Fala',
 'com',
 'o',
 'senhor',
 'Clóvis',
 'Gentili',
 '.',
 'Sim',
 ',',
 'Clóvis',
 ',',
 'me',
 'chamo',
 'Beatriz',
 '.',
 'Fala',
 'pra',
 'todos',
 '.',
 'Tudo',
 'bem',
 '?',
 'Tudo',
 'bem',
 '?',
 'Que',
 'bom',
 '!',
 'Você',
 'gostaria',
 'de',
 'falar',
 'com',
 'o',
 'senhor',
 '?',
 'O',
 'senhor',
 'seria',
 'o',
 'responsável',
 'da',
 'CGC',
 'atacadista',
 ',',
 'né',
 '?',
 'É',
 'bem',
 'rápido',
 'meu',
 'contato',
 ',',
 'senhor',
 'Clóvis',
 ',',
 'eu',
 'preciso',
 'só',
 'fazer',
 'um',
 'acompanhamento',
 'pra',
 'saber',
 'como',
 'tá',
 'a',
 'experiência',
 'e',
 'a',
 'satisfação',
 'de',
 'vocês',
 '.',
 'Esse',
 'acompanhamento',
 'dura',
 'dois',
 'minutinhos',
 'no',
 'máximo',
 '.',
 'É',
 'feito',
 'pelo',
 'método',
 'de',
 'NPS',
 '.',
 'Tudo',
 'bem',
 ',',
 'tudo',
 'bem',
 '.',
 'Tá',
 '.',
 'O',
 'senhor',
 'vai',
 'avaliar',
 'somente',
 'de',
 '0A10A',
 'marca',
 'Totem',
 '.',


* Frequência de palavras

* Mais frequentes

In [46]:
complete_text = (transcricao['transcription'])
complete_doc = nlp(complete_text)

words = [
    token.text
    for token in complete_doc
    if not token.is_stop and not token.is_punct and not token.like_num
]

print(Counter(words).most_common(5))

[('senhor', 15), ('Tá', 6), ('Clóvis', 5), ('contato', 4), ('hoje', 4)]


* Menos frequentes

In [47]:
words = [
    token.text
    for token in complete_doc
    if not token.is_stop and not token.is_punct and not token.like_num
]

print(Counter(words).most_common()[:-6:-1])

[('ótima', 1), ('Tenha', 1), ('agradeço', 1), ('retornaremos', 1), ('passando', 1)]


* Demonstração de remoção de stop words

In [48]:
word_counter = Counter(
    [token.text for token in complete_doc if not token.is_punct]
)

# Print the 5 most common words
print(word_counter.most_common(5))

[('a', 20), ('senhor', 15), ('o', 14), ('de', 12), ('do', 10)]


In [49]:
def contar_palavras(texto):
    """
    Conta a quantidade total de palavras em um texto.

    Args:
        texto: O texto a ser analisado.

    Returns:
        O número total de palavras no texto.
    """
    palavras = texto.split()
    return len(palavras)

quantidade_palavras = contar_palavras(transcricao['transcription'])
print("O texto tem", quantidade_palavras, "palavras.")

O texto tem 400 palavras.


In [50]:
# Lista de palavras mágicas
palavras_magicas = ["obrigado", "obrigada", "por favor", "desculpa", "desculpe",
                     "por gentileza", "bom dia", "boa tarde", "boa noite",
                     "agradeço", "agradece", "gratidão", "sinto muito",
                     "perdão", "me perdoe", "com licença"]

# Conta a frequência de cada palavra mágica no texto
contagem_palavras_magicas = {palavra: (transcricao['transcription']).lower().count(palavra) for palavra in palavras_magicas}

# Calcula o número total de palavras mágicas
total_palavras_magicas = sum(contagem_palavras_magicas.values())

# Calcula o percentual de palavras mágicas no texto
percentual_palavras_magicas = (total_palavras_magicas / quantidade_palavras) * 100

# Imprime o percentual
print("Percentual de palavras mágicas no texto: {:.2f}%".format(percentual_palavras_magicas))

# Imprime a frequência de cada palavra mágica
for palavra, contagem in contagem_palavras_magicas.items():
    print("- {}: {} vezes".format(palavra, contagem))

Percentual de palavras mágicas no texto: 1.25%
- obrigado: 1 vezes
- obrigada: 1 vezes
- por favor: 0 vezes
- desculpa: 0 vezes
- desculpe: 0 vezes
- por gentileza: 0 vezes
- bom dia: 0 vezes
- boa tarde: 2 vezes
- boa noite: 0 vezes
- agradeço: 1 vezes
- agradece: 0 vezes
- gratidão: 0 vezes
- sinto muito: 0 vezes
- perdão: 0 vezes
- me perdoe: 0 vezes
- com licença: 0 vezes


* Separar quantidade de frases em um arquivo

In [51]:
about_doc = nlp(transcricao['transcription'])
sentences = list(about_doc.sents)
len(sentences)

62

In [52]:
for sentence in sentences:
  print(f"{sentence[:30]}...")

Alô, boa tarde....
Boa tarde....
Fala com o senhor Clóvis Gentili....
Sim, Clóvis, me chamo Beatriz....
Fala pra todos....
Tudo bem?...
Tudo bem?...
Que bom!...
Você gostaria de falar com o senhor?...
O senhor seria o responsável da CGC atacadista, né?...
É bem rápido meu contato, senhor Clóvis, eu preciso só fazer um acompanhamento pra saber como tá a experiência e a satisfação de vocês....
Esse acompanhamento dura dois minutinhos no máximo....
É feito pelo método de NPS....
Tudo bem, tudo bem....
Tá....
O senhor vai avaliar somente de 0A10A marca Totem....
Alguns pontos que eu vou perguntar....
Tá....
Vou confirmar o CNPJ....
CNPJ 19 705 oito três zero 1000 contra quatro dois Ok....
Ok....
Obrigada pela confirmação, senhor Clóvis....
Numa escala de 0 a 10, qual a probabilidade do senhor recomendar a TOTVS a um amigo ou colega hoje?...
Tem algum comentário que o senhor gostaria de deixar sobre a nota?...
Não....
Então, para a gente aprofundar um pouco mais na melhoria, agora eu vou fa

* Delimitação

In [53]:
ellipsis_text = (transcricao['transcription'])

@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    """Add support to use `...` as a delimiter for sentence detection"""
    for token in doc[:-1]:
        if token.text == "...":
            doc[token.i + 1].is_sent_start = True
    return doc

In [54]:
custom_nlp = spacy.load("pt_core_news_lg")
custom_nlp.add_pipe("set_custom_boundaries", before="parser")
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)

Alô, boa tarde.
Boa tarde.
Fala com o senhor Clóvis Gentili.
Sim, Clóvis, me chamo Beatriz.
Fala pra todos.
Tudo bem?
Tudo bem?
Que bom!
Você gostaria de falar com o senhor?
O senhor seria o responsável da CGC atacadista, né?
É bem rápido meu contato, senhor Clóvis, eu preciso só fazer um acompanhamento pra saber como tá a experiência e a satisfação de vocês.
Esse acompanhamento dura dois minutinhos no máximo.
É feito pelo método de NPS.
Tudo bem, tudo bem.
Tá.
O senhor vai avaliar somente de 0A10A marca Totem.
Alguns pontos que eu vou perguntar.
Tá.
Vou confirmar o CNPJ.
CNPJ 19 705 oito três zero 1000 contra quatro dois Ok.
Ok.
Obrigada pela confirmação, senhor Clóvis.
Numa escala de 0 a 10, qual a probabilidade do senhor recomendar a TOTVS a um amigo ou colega hoje?
Tem algum comentário que o senhor gostaria de deixar sobre a nota?
Não.
Então, para a gente aprofundar um pouco mais na melhoria, agora eu vou falar os pontos específicos e peço que o senhor avalie também somente de 0 a 

* Posicionamento/Index do Token

In [55]:
about_text = (transcricao['transcription'])

about_doc = nlp(about_text)

for token in about_doc:
    print(token, token.idx)

Alô 0
, 3
boa 5
tarde 9
. 14
Boa 16
tarde 20
. 25
Fala 27
com 32
o 36
senhor 38
Clóvis 45
Gentili 52
. 59
Sim 61
, 64
Clóvis 66
, 72
me 74
chamo 77
Beatriz 83
. 90
Fala 92
pra 97
todos 101
. 106
Tudo 108
bem 113
? 116
Tudo 118
bem 123
? 126
Que 128
bom 132
! 135
Você 137
gostaria 142
de 151
falar 154
com 160
o 164
senhor 166
? 172
O 174
senhor 176
seria 183
o 189
responsável 191
da 203
CGC 206
atacadista 210
, 220
né 222
? 224
É 226
bem 228
rápido 232
meu 239
contato 243
, 250
senhor 252
Clóvis 259
, 265
eu 267
preciso 270
só 278
fazer 281
um 287
acompanhamento 290
pra 305
saber 309
como 315
tá 320
a 323
experiência 325
e 337
a 339
satisfação 341
de 352
vocês 355
. 360
Esse 362
acompanhamento 367
dura 382
dois 387
minutinhos 392
no 403
máximo 406
. 412
É 414
feito 416
pelo 422
método 427
de 434
NPS 437
. 440
Tudo 442
bem 447
, 450
tudo 452
bem 457
. 460
Tá 462
. 464
O 466
senhor 468
vai 475
avaliar 479
somente 487
de 495
0A10A 498
marca 504
Totem 510
. 515
Alguns 517
pontos 524
que 531

* Classificação da palavra

In [56]:
print(
    f"{'Text with Whitespace':22}"
    f"{'Is Alphanumeric?':15}"
    f"{'Is Punctuation?':18}"
    f"{'Is Stop Word?'}"
)

for token in about_doc:
    print(
        f"{str(token.text_with_ws):22}"
        f"{str(token.is_alpha):15}"
        f"{str(token.is_punct):18}"
        f"{str(token.is_stop)}"
    )

Text with Whitespace  Is Alphanumeric?Is Punctuation?   Is Stop Word?
Alô                   True           False             False
,                     False          True              False
boa                   True           False             True
tarde                 True           False             True
.                     False          True              False
Boa                   True           False             True
tarde                 True           False             True
.                     False          True              False
Fala                  True           False             False
com                   True           False             True
o                     True           False             True
senhor                True           False             False
Clóvis                True           False             False
Gentili               True           False             False
.                     False          True              False
Sim                  

* Removendo stop words
* Removendo pontuações


In [57]:
custom_about_text = (transcricao['transcription'])

nlp = spacy.load("pt_core_news_lg")
about_doc = nlp(custom_about_text)

print([token for token in about_doc if not token.is_stop and not token.is_punct])

[Alô, Fala, senhor, Clóvis, Gentili, Clóvis, chamo, Beatriz, Fala, pra, gostaria, falar, senhor, senhor, responsável, CGC, atacadista, né, rápido, contato, senhor, Clóvis, preciso, acompanhamento, pra, tá, experiência, satisfação, acompanhamento, dura, minutinhos, feito, método, NPS, Tá, senhor, avaliar, 0A10A, marca, Totem, vou, perguntar, Tá, Vou, confirmar, CNPJ, CNPJ, 19, 705, 1000, Ok, Ok, confirmação, senhor, Clóvis, escala, 0, 10, probabilidade, senhor, recomendar, TOTVS, amigo, colega, hoje, algum, comentário, senhor, gostaria, deixar, nota, gente, aprofundar, melhoria, vou, falar, específicos, peço, senhor, avalie, 0, 10, caso, tenha, contato, Tá, falando, suporte, técnico, senhor, avalia, agilidade, processos, suporte, hoje, demorado, 0, 10, Tá, falando, atendimento, agentes, suporte, né, Falando, comercial, atendimento, executivo, vendas, Falando, custos, valores, produtos, contratados, Atendimento, financeiro, referente, boletos, negociações, caso, precisem, precisamos, col

* Diferentes categorizações de palavras

In [58]:
lemma_text = (transcricao['transcription'])

lemma_doc = nlp(lemma_text)

for token in lemma_doc:
    if str(token) != str(token.lemma_):
        print(f"{str(token):>20} : {str(token.lemma_)}")

                 Alô : alô
                 boa : bom
                Fala : falar
                 Sim : sim
                  me : eu
               chamo : chamar
                Fala : falar
               todos : todo
                Tudo : tudo
                Tudo : tudo
                 Que : que
                Você : você
            gostaria : gostar
                   O : o
               seria : ser
                  da : de o
                   É : ser
                   a : o
                   a : o
               vocês : você
                Esse : esse
                dura : durar
          minutinhos : minutinho
                  no : em o
                   É : ser
               feito : fazer
                pelo : por o
                Tudo : tudo
                   O : o
                 vai : ir
              Alguns : algum
              pontos : ponto
                 vou : ir
                  Ok : ok
            Obrigada : obrigado
                pela : por 

* Demonstração de como uma palavra está sendo usada em uma frase

In [59]:
about_text = (transcricao['transcription'])

about_doc = nlp(about_text)

for token in about_doc:
    print(
        f"""
        TOKEN: {str(token)}
        =====
        TAG: {str(token.tag_):10} POS: {token.pos_}
        EXPLANATION: {spacy.explain(token.tag_)}"""
    )


        TOKEN: Alô
        =====
        TAG: INTJ       POS: INTJ
        EXPLANATION: interjection

        TOKEN: ,
        =====
        TAG: PUNCT      POS: PUNCT
        EXPLANATION: punctuation

        TOKEN: boa
        =====
        TAG: ADJ        POS: ADJ
        EXPLANATION: adjective

        TOKEN: tarde
        =====
        TAG: NOUN       POS: NOUN
        EXPLANATION: noun

        TOKEN: .
        =====
        TAG: PUNCT      POS: PUNCT
        EXPLANATION: punctuation

        TOKEN: Boa
        =====
        TAG: ADJ        POS: ADJ
        EXPLANATION: adjective

        TOKEN: tarde
        =====
        TAG: ADV        POS: ADV
        EXPLANATION: adverb

        TOKEN: .
        =====
        TAG: PUNCT      POS: PUNCT
        EXPLANATION: punctuation

        TOKEN: Fala
        =====
        TAG: VERB       POS: VERB
        EXPLANATION: verb

        TOKEN: com
        =====
        TAG: ADP        POS: ADP
        EXPLANATION: adposition

        TOKEN:

# Pré-processamento

In [60]:
complete_text = (transcricao['transcription'])

complete_doc = nlp(complete_text)

def is_token_allowed(token):
    return bool(
        token
        and str(token).strip()
        and not token.is_stop
        and not token.is_punct
    )

def preprocess_token(token):
    return token.lemma_.strip().lower()

complete_filtered_tokens = [
    preprocess_token(token)
    for token in complete_doc
    if is_token_allowed(token)
]

In [61]:
complete_filtered_tokens

['alô',
 'falar',
 'senhor',
 'clóvis',
 'gentili',
 'clóvis',
 'chamar',
 'beatriz',
 'falar',
 'pra',
 'gostar',
 'falar',
 'senhor',
 'senhor',
 'responsável',
 'cgc',
 'atacadista',
 'né',
 'rápido',
 'contato',
 'senhor',
 'clóvis',
 'preciso',
 'acompanhamento',
 'pra',
 'tá',
 'experiência',
 'satisfação',
 'acompanhamento',
 'durar',
 'minutinho',
 'fazer',
 'método',
 'nps',
 'tá',
 'senhor',
 'avaliar',
 '0a10a',
 'marca',
 'totem',
 'ir',
 'perguntar',
 'tá',
 'vou',
 'confirmar',
 'cnpj',
 'cnpj',
 '19',
 '705',
 '1000',
 'ok',
 'ok',
 'confirmação',
 'senhor',
 'clóvis',
 'escala',
 '0',
 '10',
 'probabilidade',
 'senhor',
 'recomendar',
 'totvs',
 'amigo',
 'colega',
 'hoje',
 'algum',
 'comentário',
 'senhor',
 'gostar',
 'deixar',
 'nota',
 'gente',
 'aprofundar',
 'melhoria',
 'ir',
 'falar',
 'específico',
 'peçer',
 'senhor',
 'avaliar',
 '0',
 '10',
 'caso',
 'ter',
 'contato',
 'tá',
 'falar',
 'suporte',
 'técnico',
 'senhor',
 'avaliar',
 'agilidade',
 'processo'

* Palavras linkadas e suas dependências

In [62]:
denp_text = transcricao['transcription']
denp_doc = nlp(denp_text)

for token in denp_doc:
    print(
        f"""
        TOKEN: {token.text}
        =====
        {token.tag_ = }
        {token.head.text = }
        {token.dep_ = }
        """
    )


        TOKEN: Alô
        =====
        token.tag_ = 'INTJ'
        token.head.text = 'Alô'
        token.dep_ = 'ROOT'
        

        TOKEN: ,
        =====
        token.tag_ = 'PUNCT'
        token.head.text = 'Alô'
        token.dep_ = 'punct'
        

        TOKEN: boa
        =====
        token.tag_ = 'ADJ'
        token.head.text = 'tarde'
        token.dep_ = 'amod'
        

        TOKEN: tarde
        =====
        token.tag_ = 'NOUN'
        token.head.text = 'Alô'
        token.dep_ = 'dep'
        

        TOKEN: .
        =====
        token.tag_ = 'PUNCT'
        token.head.text = 'Alô'
        token.dep_ = 'punct'
        

        TOKEN: Boa
        =====
        token.tag_ = 'ADJ'
        token.head.text = 'tarde'
        token.dep_ = 'amod'
        

        TOKEN: tarde
        =====
        token.tag_ = 'ADV'
        token.head.text = 'tarde'
        token.dep_ = 'ROOT'
        

        TOKEN: .
        =====
        token.tag_ = 'PUNCT'
        token.he

In [63]:
client.close()

In [64]:
# https://realpython.com/natural-language-processing-spacy-python/